# 3 监督学习
监督学习的目标是通过构建自变量与因变量间关系的拟合模型，解释数据的规律性，以便预测未来的结果；即：当 $y$ 已知时，求解函数 $F$ ，使 $y = F(x)$。现实情境中相等很难，尽可能拟合训练数据中大多数 $y$ 点。根据 $y$ 的差异划分为处理**回归**（$y$ 无限集）与**分类**（$y$ 有限集）两种问题，多数监督学习算法都同时支持回归与分类问题。

分类问题可再分为二分类与多分类问题，有些算法（如随机森林、朴素贝叶斯等）可同时支持处理上述细分类别，但也存在部分严格的二分类器（线性分类器、SVM等），可使用如下策略：
- **一对所有（OvA）**：创建与类别数目相等的 $N$ 个分类器，单个分类器训练时，仅选取单一类别为正例其余均为负例，决策时选取决策分数最高的分类器所对应的类。
- **一对一（OvO）**：将类别数量 $N$ 两两配对作为训练子集单独训练二分类器，共计训练 $\frac{N*(N-1)}{2}$ 个分类器，决策时选出全部分类器中胜出最多的类（比如投票机制）。

根据 $F$ 的差异划分：
- **参数模型**：预设数据服从某个分布（$F$），目标是评估数据分布的参数值，如线性回归、逻辑回归、感知机等
- **非参数模型**：预设数据分布存在但未知（参数可变且无数量上限），只能通过非参数统计的方法进行推断，如knn、决策树、朴素贝叶斯、支持向量机、神经网络等

如前述，数据建模的三要素：模型假设、损失函数、寻解算法，本章下面开始将模型的差异抹平，统一符号表示，以聚焦探讨损失函数与寻解算法。

## 3.1 参数模型
### 3.1.1 回归问题的损失函数
损失函数，即模型达成的目标，回归问题的目标必然是希望模型的预测值 $\hat y$ 与真值 $y$ 越接近越好（最佳情况一摸一样），即：$y - \hat y$ 越小越好，且希望拟合结果覆盖大多数训练数据，故还需兼顾均衡性：
$$ MBE(平均偏差误差) = \frac{1}{n}\sum_{i = 1}^n(y_i - \hat y_i),n为训练记录数 $$
从公式中不难发现，MBE会受到误差方向的影响，产生正负相互抵消，评估损失的精确度不足的问题，进行简单改良：
$$ MAE(平均绝对值误差,L1Loss) = \frac{1}{n}\sum_{i = 1}^n|y_i - \hat y_i| $$
MAE虽然改善了正负偏差相互抵消的问题，但是函数在零点不可导，为后续寻找全局最优解造成了困扰，故再次优化：
$$ RMSE(均方根误差,L2Loss) = \sqrt {MSE(均方误差)} = \sqrt {\frac{1}{n}\sum_{i = 1}^n(y_i - \hat y_i)^2} $$
RMSE函数具有处处可导的特性，相比于内层的MSE，数值量级下降很多，使其成为了最常使用的回归问题损失函数；但是指数特性让该损失函数对离群点极其敏感，会刻意牺牲对正常点的优化，着重优化对贡献更高的离群点，所以需多加注意；当然也有针对于此的改良方案：
$$ HuberLoss(平滑绝对误差) = \sum_{i=1}^n \begin{cases} \frac{1}{2}(y_i - \hat y_i)^2 & |y_i - \hat y_i|<\delta \\ \delta * (|y_i - \hat y_i|-0.5 \delta ^2) & else \end{cases}$$
当预测值与真值差别过大（大于 $\delta$ ）时，使用L1范数，其余情况使用L2范数，可平衡异常点对损失函数值的影响，该函数最常用的形态是：
$$ smooth_{L1}(\delta = 1) = \sum_{i=1}^n \begin{cases} 0.5*(y_i - \hat y_i)^2 & |y_i - \hat y_i|<1 \\ |y_i - \hat y_i|-0.5 & else \end{cases} $$
或者使用如下改良方案：
$$ LogCoshLoss = \sum_{i=1}^n log(cosh(y_i - \hat y_i)) $$
对于较小的 $x$，$log(cosh(x))$ 近似于 $\frac{1}{2}x^2$，对于较大的 $x$，近似于$|x|-log(2)$，具有Huber loss所有的优点同时满足二阶导数处处可微；但也存在误差极大时，一阶导数为定值的问题。

最后介绍一个有一定适用场景的变体损失函数：
$$ QuantileLoss(分位数损失) = \sum_{y_i<\hat y_i}(1-\lambda)|y_i-\hat y_i| + \sum_{y_i\geq\hat y_i}\lambda|y_i - \hat y_i|, \lambda \in [0,1] $$
适用于偏重对某区间严苛拟合，容忍其余部分精准度有一定偏差的问题，通过对正反误差的重视程度，对$\lambda$（分位值）给予不同的惩罚，让中值产生偏移，突出重点区间。

以上只是我熟识的回归问题损失函数，当然还有许多未提及，亦可根据实际问题自定义。

#### 3.1.2 分类问题的损失函数
对于分类问题，使用RMSE及其相关变体理论上也是可以的，但是部分模型叠加了符号函数，可能造成损失函数存在密集的局部最优解，难以寻找全局最优解；故介绍几个比较适合分类问题的损失函数。
#### 3.1.2.1 负似然对数
负似然对数，即：**似然函数**的负值，以二分类问题为例，讨论似然函数，假设如下：

1. 假设事件有且仅有互斥的正反两种结果；正例记为1，负例记为0。
2. 假设事件正例发生的概率为：$p$，则反例概率为：$1-p$。
3. 对事件累计实验 $N$ 次，其中正例 $m$ 次，反例 $n$ 次。

则 $N$ 次实验发生上述结果的可能性为：$L(p)=p^m*(1-p)^n$（实验结果存在有序性，如无序数据则为：$L(p)=C^m_N*p^m*(1-p)^n$）这就是似然函数，其结果称为似然率，表征事件发生的可能性。接着将 $p$ 替换为模型 $h_\theta(x)$ 的预测结果：
$$ p(\theta) = \begin{cases} h_\theta(x) & y=1 \\ 1-h_\theta(x) & y=0 \end{cases} = h_\theta(x)^y(1-h_\theta(x))^{(1-y)},y为正反例真值 $$
将 $p(\theta)$ 代入似然函数：
$$ L(\theta) = \prod_{i=1}^n h_\theta(x)^{y_i}(1-h_\theta(x))^{(1-y_i)}  $$
依据频率学派的观点，充分多的已知事件结果的似然率最高，故使 $L(p)$ 取最大值的 $\hat p$ 便是 $p$，这样就将求解函数参数的问题等价于寻找合适的 $ \theta $ 使似然函数取最大值；对应于负似然函数就是求解最小值，可作损失函数：
$$ cost(\theta) = -L(\theta) = - \prod_{i=1}^n h_\theta(x)^{y_i}(1-h_\theta(x))^{(1-y_i)} $$
对上式直接求导颇为复杂，引入对数简化复杂度，不改变单调性及最小值点取值：
$$ cost(\theta) = -lnL(\theta) = - \sum_{i=1}^n y_i * ln(h_\theta(x)) + (1-y_i)*ln(1-h_\theta(x)) $$
矩阵形式为：
$$ cost(\theta) = -Y^T * ln(h_\theta(X))-(E - Y)^T * (E - ln(h_\theta(X))) $$

#### 3.1.2.2 交叉信息熵
因个人水平不足以理论化介绍交叉信息熵(cross entropy)，故下面用粗浅的认知进行阐述，准确的信息可查阅信息论相关文档。

**信息**，可以说是一种价值度量。以新闻事件来类比，超出人们想象的、小概率发生的事件我们才会将其作为头条新闻，因为该新闻的价值高；类似的，一个系统中，小概率事件蕴含的信息值高。怎么度量呢？因为信息起源于计算机对信息的记录，只有0/1两种标记方案，故以二分类决策推测某事件的具体结果为类比，最优策略总是优先选择最可能发生的结果，此后逐层递减。这个准则下，确定某个事件结果发生所消耗的路径长度就是信息，计算公式：$log_2(\frac{1}{p})$，$p$ 即为某事件的概率（此处我认为$log_n(\frac{1}{p})$,$n$为分叉数）。

**信息熵**：分类系统中多个类别信息值的加权和：$ \sum_{i=1}^n p_i * log_2(\frac{1}{p_i}) $。

**交叉熵**：基于模型预测的事件概率分布$q_i$的信息加权和：$ \sum_{i=1}^n p_i * log_2(\frac{1}{q_i}) $。

**相对熵**：$ 相对熵 = 交叉熵 - 信息熵 = \sum_{i=1}^n p_i * log_2(\frac{p_i}{q_i})$

所以损失函数可以使用相对熵：$ cost(\theta) = \sum_{i=1}^n p_i * log_2(\frac{1}{q_i}) - \sum_{i=1}^n p_i * log_2(\frac{1}{p_i}) = -\sum_{i=1}^n p_i * log_2(q_i) + \sum_{i=1}^n p_i * log_2(p_i) $,将 $h_\theta(x)$ 带入公式并优化常数项 $\sum_{i=1}^n p_i * log_2(p_i)$：
$$ cost(\theta) = -\sum_{i=1}^n p_i * log_2(q_i) = -\sum_{i=1}^n p_i * [y_i * log_2(h_\theta(x)) + (1-y_i) * log_2(1 - h_\theta(x))] $$
*这里我枚举测验过只有当 $p_i = q_i$ 时，$- \sum_{i=1}^n p_i log_2(\frac{1}{q_i})$ 最小，具体证明我还不会。*

#### 3.1.2.3 指数损失(exponential loss)
*暂时还没研究透彻，adaboost的损失函数*

#### 3.1.2.4 Hinger损失
*暂时还没研究，感知机/支持向量机的损失函数*

#### 3.1.2.5 铰链损失
*暂时还没研究*

### 3.1.3 寻解算法：最小二乘法（OLS）
**最小二乘法（最小平方法）**：通过最小化RMSE，寻找模型的最佳匹配参数；求解最小二乘法结果的方法有**矩阵解法**和**梯度下降法**。

*以前我以为最小二乘法和梯度下降法是两个方法，其实梯度下降法只是最小二乘法求解的方法之一。*

#### 3.1.3.1 矩阵解法
为方便后续讲解，设定前提假设如下：
- **模型假设**：$ F(x)=a_1*x_1+a_2*x_2+……+a_n*x_n+b=[x_1,x_2,……,x_n,1] \cdot [a_1,a_2,……,a_n,b]^T=XA $
- **损失函数**：$ L(x,a)=\frac{1}{2n}\sum_{i=1}^n[(a_1*x_{i1}+a_2*x_{i2}+……+a_n*x_{in}+b)-y_i]^2=\frac{1}{2n}(XA-Y)^T(XA-Y) $

依据假设，当损失函数的一阶偏导数（$\partial A$）为零时，损失函数取最小值，即：
$$ \frac{\partial L}{\partial A}=\frac{\partial L}{\partial a_1}+\frac{\partial L}{\partial a_2}+……+\frac{\partial L}{\partial a_n}=\frac{1}{n}\sum_{i=1}^n(a_1x_{i1}+a_2x_{i2}+……+a_nx_{in}+b-y_i)*(x_{i1}+x_{i2}+……+x_{in})=\frac{X^T(XA-Y)}{n}=0 $$
求解：
$$ A = (X^TX)^{-1}X^TY $$
以上的推导过程表明矩阵解法有如下限制：

1. 矩阵 $X$ 必须存在逆矩阵（舍弃边缘特征）。
2. 矩阵 $X$ 的求逆过程的耗时受矩阵规模影响（降维处理）。
3. 模型假设若非线性，求解将十分复杂或难以开展（转化线性关系）。

*（）中是一些预处理方案，优化矩阵的限制。*

#### 3.1.3.2 梯度下降法
梯度下降法是利用梯度逐步迭代模型参数，求解损失函数最小值的方法。
```
想象你和贝爷一起进行丛林探险，露宿山崖边缘，晨间丛林里雾气弥漫，你们选择下至山下的峡谷补充水源。此时，你们**各选了一个位置**开始下山，你用脚一点点感受被踩踏的岩石，选择合适的落脚点，你总是**优先选择垂直向下（即坡度最陡）**的方向下山，重复如此，最终你们下到了山脚。
```
梯度下降法的过程与之相似，前提假设如下：
- **模型假设**：$ F(x)=XA $ 并随机初始化$A$中参数。
- **损失函数**：$ L(x,a)=\frac{1}{2n}(XA-Y)^T(XA-Y) $。
- **偏导数**：$ \frac{\partial L}{\partial A}=\frac{1}{n}X^T(XA-Y) $。
- **梯度下降**：$ A=A-\alpha \frac{\partial L}{\partial A},\alpha \in (0,1) $。
- **终止条件**：损失不可改善或达预设终止条件时。

学习率（$\alpha$）与梯度（$\frac{\partial L}{\partial A}$）共同决定每轮参数的迭代步长，其中学习率是用户设定值，需注意：
- 模型训练前特征需缩放，提高 $\alpha$ 作用效果。
- $\alpha$ 设定过小，耗时严重；设定过大，反复跳跃难收敛。

与矩阵解法中求解一阶导数为零不同，由于训练数据 $X$，随机参数 $A$、学习率 $\alpha$ 均为已知项，其目标是将现有的随机参数不断改善成最优函数的接近值，所以突破了矩阵解法的限制，使其适用于包含RMSE在内的多种损失函数，但有些损失函数除了全局最小值外，还存在多个局部最小值，有时可能永远到不了全局最小值，值得注意。

实际使用时衍生出三种方法：

1. **批梯度下降（BGD）**：每轮迭代计算梯度时使用全部训练数据；优点是朝着最小值迭代，缺点是样本值很大时每轮更新速度会很慢。
2. **随机梯度下降（SGD）**：每轮迭代计算梯度时随机抽取一条训练数据；优点是每轮更新速度快，缺点是样本噪点较多时未必朝着极小值方向更新（多轮迭代能保障整体大致朝着极小值方向更新）。
3. **小批量梯度下降（MBGD）**：每轮迭代计算梯度时随机抽取一定比例训练数据；优点是平衡更新梯度方向准确性与更新速度，缺点是batch值较难确定。

随机性可以很好的跳过局部最优值，所以比较推荐使用小批量梯度下降法；对于随机梯度下降较难快速达到最小值问题，可使用逐渐降低学习率改善，这个过程被称为模拟退火。

上述虽然只使用了简单的回归问题举例说明，但是分类问题也适用（负似然对数与交叉信息熵的梯度形式：$X^T*(h_\theta(x)-Y)$），亦可推广到更高维，更复杂的损失函数中使用。

## 3.2 非参数模型
非参只是数据分布不可知，不代表建模本身没有参数需要调整，我们还是要通过控制推断参数，进而控制模型的拟合情况。

### 3.2.1 knn
knn是一个简单实用、基于实例学习的算法，其原理是：

1. 以选定的建模特征构建多维空间
2. 计算预测数据与样本数据各点间的空间距离
3. 将样本点按距离由近到远排序
4. 挑选前 $k$ 个样本数据点，作为决策基础
5. 将 $k$ 个样本所属类别中多数类作为预测类

算法的核心（需调参）是 $k$ 的具体数量和距离的测算方法（欧氏距离、曼哈顿距离等）。knn受限于需要一定的存储空间保存训练数据，无法给出模型结构，计算耗时长等问题。

### 3.2.2 朴素贝叶斯
朴素贝叶斯是基于概率论的分类方法，基于不同的概率论原理划分为：
- 伯努利实验：注重0-无/1-有，不考虑次数权重，常见的是词集模型
- 多项式实验：考虑次数权重，常见的是词袋模型

其核心是**条件概率**、**贝叶斯准则**和**全概率公式**：
- 条件概率：$P(X|Y) = \frac{P(XY)}{P(Y)}$
- 贝叶斯准则：$P(Y|X) = \frac{P(X|Y) * P(Y)}{P(X)}$
- 全概率公式：$P(X) = \sum_{i=1}^n P(X|y_i)P(y_i)$，其中 $P(y_i)$ 互斥且 $\sum_{i=1}^n P(y_i) = 1$
- $P(Y=y_k|X) = \frac{P(X|y_k) * P(y_k)}{\sum_{i=1}^n P(X|y_i)P(y_i)}$，其中 $n$ 代表 $Y$ 可以取值的类别，$y_k$表示其中某个类别
- $P(Y=y_k|X)$ 的计算难点是 $P(X|y_k)$
- 朴素贝叶斯假设特征之间相互独立，则 $P(X|y_k) = P(x_1|y_k) * P(x_2|y_k) * …… * P(x_n|y_k)$
- 特征独立性的条件现实中往往很难满足，但通过牺牲准确性换取计算简化，实际效果还是不错的

除了独立性外，$P(x_i|y_k)$ 存在隐患：

1. 局部值为零，影响最终预测结果
2. 局部值极小，计算机最终结果输出零

针对上述问题，解决方案如下：

1. **拉普拉斯变换**：$ \frac{N_k+\lambda}{N+O_k \lambda},\lambda \in [0,1] $
2. 对数计算：$ P(x_1|y_k) * P(x_2|y_k) * …… * P(x_n|y_k) = e^{ln(P(x_1|y_k))+ln(P(x_2|y_k))+……+ln(P(x_n|y_k))} $，乘法变加法

### 3.2.3 决策树
决策树是一个具有高度可解释性的模型，基于特征阈值分割数据，每轮分割将原始数据划分为单一类别占比更高的子集，直到子集中分类近乎为单一类别，主流的算法有ID3、C4.5及CART。

### 3.2.3.1 ID3 & C4.5
ID3和C4.5都是基于信息论进行不纯度（单一类别占比情况）度量的，且C4.5是ID3的改进方法，故先介绍ID3算法。回顾交叉信息熵中信息熵公式：$ \sum_{i=1}^n p_i * log_2(\frac{1}{p_i}) $，当 $p_i = 0.5$ 时，信息熵取最大值；当 $p_i = 1$ 时，信息熵取最小值0；可以作为度量数据集合纯净度的指标，ID3算法便是用此理念来评估数据分割后的优劣程度。

以二分类决策树举例，看看ID3算法是怎么实施的：

1. 决策树中某待分裂节点记为：$a_o$，节点中数据记录数记为：$N_o$，信息熵记为：$ -\sum_{i=1}^n p_{oi} * log(P_{oi}),n为类别数量$。
2. 待分裂节点分裂后的左支子节点记为：$a_l$，节点中数据记录数记为：$N_l$，信息熵记为：$ -\sum_{i=1}^n p_{li} * log(P_{li}),n为类别数量$。
3. 待分裂节点分裂后的右支子节点记为：$a_r$，节点中数据记录数记为：$N_r$，信息熵记为：$ -\sum_{i=1}^n p_{ri} * log(P_{ri}),n为类别数量$。
4. 迭代各个特征及其阈值记录分裂前后系统信息熵的差异：$ -\sum_{i=1}^n p_{oi} * log(P_{oi}) - (-\frac{N_l}{N_o}\sum_{i=1}^n p_{li} * log(P_{li})) - (-\frac{N_r}{N_o}\sum_{i=1}^n p_{ri} * log(P_{ri}))$，称作信息增益。
5. 选取信息增益最大的特征及其阈值作为分割节点的规则，分割决策树。
6. 将新分割的节点循环1-5步骤，直至特征使用完毕或者决策树无法再次进行分割。

ID3算法存在几个缺陷：

1. 信息增益准则对可取值数目较多的特征有所偏好，类似“编号”的特征其信息增益接近于1。
2. 没有剪枝策略，模型容易过拟合。
3. 算法本身没有二分叉树的假设，可以多分叉，连续值处理较麻烦。
4. 没有考虑缺失值。

针对问题1，C4.5作为ID3的改进算法，一定程度上缓解了信息增益受可取值数量的影响，引入“属性熵”的概念。所谓“属性熵”是将子节点看作系统整体，每个节点看作不同的分类，如ID3的示例中 $a_l$ 与 $a_r$ 的“属性熵”为：$ -\frac{N_l}{N_o}*log(\frac{N_l}{N_o}) + (-\frac{N_r}{N_o}*log(\frac{N_r}{N_o}) ) $，那么C4.5就将原来ID3中信息增益作为判断分裂特征及其阈值选择的评价标准转换为：$\frac{信息增益}{属性熵}$，称作信息增益率。

针对问题2，引入悲观剪枝策略进行后剪枝；
针对问题3，将连续特征离散化，假设 n 个样本的连续特征 A 有 m 个取值，C4.5 将其排序并取相邻两样本值的平均数共 m-1 个划分点，分别计算以该划分点作为二元分类点时的信息增益，并选择信息增益最大的点作为该连续特征的二元离散分类点；

针对问题4，对于缺失值的处理可以分为两个子问题：

1. 在特征值缺失的情况下进行划分特征的选择？（即如何计算特征的信息增益率）:对于具有缺失值特征，用没有缺失的样本子集所占比重来折算；
2. 选定该划分特征，对于缺失该特征值的样本如何处理？（即到底把这个样本划分到哪个结点里）：将样本同时划分到所有子节点，不过要调整样本的权重值，其实也就是以不同概率划分到不同节点中。


### 3.2.2.2 CART
基尼不纯度，其实截取的熵的泰勒展开项

### 3.2.4 支持向量机
支持向量
SVM模型有两个非常重要的参数C与gamma。其中 C是惩罚系数，即对误差的宽容度。c越高，说明越不能容忍出现误差,容易过拟合。C越小，容易欠拟合。C过大或过小，泛化能力变差
gamma是选择RBF函数作为kernel后，该函数自带的一个参数。隐含地决定了数据映射到新的特征空间后的分布，gamma越大，支持向量越少，gamma值越小，支持向量越多。支持向量的个数影响训练与预测的速度。

## 3.3 评估模型

## 3.4 交叉验证
模型调参神器

---
参考资料：
- [Sklearn 与 TensorFlow 机器学习实用指南](https://www.bookstack.cn/books/hands_on_Ml_with_Sklearn_and_TF)
- [最小二乘法](https://zhuanlan.zhihu.com/p/38128785)
- [极大似然估计与最大后验概率估计](https://zhuanlan.zhihu.com/p/40024110)
- [信息论](https://zhuanlan.zhihu.com/p/36385989)
- [理解朴素贝叶斯分类的拉普拉斯平滑](https://zhuanlan.zhihu.com/p/26329951)
- [如何通俗的解释交叉熵与相对熵](https://www.zhihu.com/question/41252833)
- [波士顿房价预测任务](https://www.paddlepaddle.org.cn/tutorials/projectdetail/1515038#anchor-13)